In [4]:
%reload_ext autoreload

"""
This script runs classifier training over the entire training data and then
output predictions over the interactome.

Usage:
  build_data.py [--interpro] [--pfam] [--mf] [--cc] [--bp]
                [--update_features] [--update_mapping]
                [--induce] [--verbose] [--n_jobs=J] [--backend=B]
  build_data.py -h | --help

Options:
  -h --help     Show this screen.
  --interpro    Use interpro domains in features.
  --pfam        Use Pfam domains in features.
  --mf          Use Molecular Function Gene Ontology in features.
  --cc          Use Cellular Compartment Gene Ontology in features.
  --bp          Use Biological Process Gene Ontology in features.
  --induce      Use ULCA inducer over Gene Ontology.
  --verbose     Print intermediate output for debugging.
  --n_jobs=J            Number of processes to run in parallel [default: 1]
  --backend=B           Python concurrent backend [default: multiprocessing]
  --update_features     Delete old feature cache and create a new one.
  --update_mapping      Delete old accession mapping and create a new one.
"""

import os
import pandas as pd

from pyppi.base import PPI, parse_args
from pyppi.data import bioplex_network_path, pina2_network_path
from pyppi.data import bioplex_v4, pina2, innate_curated, innate_imported
from pyppi.data import innate_i_network_path, innate_c_network_path
from pyppi.data import interactome_network_path, full_training_network_path
from pyppi.data import kegg_network_path, hprd_network_path
from pyppi.data import load_uniprot_accession_map, save_uniprot_accession_map
from pyppi.data import testing_network_path, training_network_path
from pyppi.data import save_network_to_path
from pyppi.data import save_ptm_labels
from pyppi.data import ppi_features_path, accession_features_path
from pyppi.data import annotation_extractor_path
from pyppi.data import pickle_pd_object

from pyppi.data_mining.features import AnnotationExtractor
from pyppi.data_mining.generic import bioplex_func, mitab_func, pina_func
from pyppi.data_mining.generic import generic_to_dataframe
from pyppi.data_mining.hprd import hprd_to_dataframe
from pyppi.data_mining.tools import process_interactions, LABEL
from pyppi.data_mining.tools import remove_intersection, remove_labels
from pyppi.data_mining.tools import map_network_accessions
from pyppi.data_mining.uniprot import UniProt, get_active_instance
from pyppi.data_mining.kegg import download_pathway_ids, pathways_to_dataframe

# -------------------------------------------------------------------------- #
#                     MODIFY THESE TO SUIT YOUR NEEDS
# -------------------------------------------------------------------------- #
args = {
    'n_jobs': 1,
    'induce': True,
    'verbose': True,
    'selection': [
        UniProt.data_types().GO_MF.value,
        UniProt.data_types().GO_BP.value,
        UniProt.data_types().GO_CC.value,
        UniProt.data_types().INTERPRO.value,
        UniProt.data_types().PFAM.value
    ],
    'update_features': True,
    'update_mapping': True,
    'backend': 'multiprocessing'
}
n_jobs = args['n_jobs']
induce = args['induce']
verbose = args['verbose']
selection = args['selection']
update_features = args['update_features']
update_mapping = args['update_mapping']
backend = args['backend']
# -------------------------------------------------------------------------- #

print("Downloading from KEGG and loading UniProt database instance...")
pathways = download_pathway_ids('hsa')
uniprot = get_active_instance(verbose=True)
data_types = UniProt.data_types()

First time loading on UniProt instance. Make take a few moments


In [5]:
# Construct all the networks
print("Building KEGG interactions...")
kegg = pathways_to_dataframe(
    pathway_ids=pathways,
    map_to_uniprot=True,
    drop_nan=True,
    allow_self_edges=True,
    allow_duplicates=False
)

Building KEGG interactions...


In [9]:
print("Building HPRD interactions...")
hprd = hprd_to_dataframe(
    drop_nan=True,
    allow_self_edges=True,
    allow_duplicates=False
)

Building HPRD interactions...


In [10]:
print("Building Interactome interactions...")
bioplex = generic_to_dataframe(
    f_input=bioplex_v4(),
    parsing_func=bioplex_func,
    drop_nan=True,
    allow_self_edges=True,
    allow_duplicates=False
)

Building Interactome interactions...


In [11]:
pina2 = generic_to_dataframe(
    f_input=pina2(),
    parsing_func=pina_func,
    drop_nan=True,
    allow_self_edges=True,
    allow_duplicates=False
)

In [12]:
innate_c = generic_to_dataframe(
    f_input=innate_curated(),
    parsing_func=mitab_func,
    drop_nan=True,
    allow_self_edges=True,
    allow_duplicates=False
)

In [13]:
innate_i = generic_to_dataframe(
    f_input=innate_imported(),
    parsing_func=mitab_func,
    drop_nan=True,
    allow_self_edges=True,
    allow_duplicates=False
)

In [22]:
print("Mapping to most recent uniprot accessions...")
# Get a set of all the unique uniprot accessions
networks = [kegg, hprd, bioplex, pina2, innate_i, innate_c]
sources = set(p for df in networks for p in df.source.values)
targets = set(p for df in networks for p in df.target.values)
accessions = list(sources | targets)

if update_mapping:
    accession_mapping = uniprot.batch_map(accessions)
    save_uniprot_accession_map(accession_mapping)
else:
    try:
        accession_mapping = load_uniprot_accession_map()
    except IOError:
        accession_mapping = uniprot.batch_map(accessions)
        save_uniprot_accession_map(accession_mapping)

Mapping to most recent uniprot accessions...
Record for Q9WUL8 not found.
Record for Q61066 not found.
Record for Q920P3 not found.
Record for Q7TQN3 not found.
Record for Q21446 not found.
Record for A2AJL3 not found.
Record for P27784 not found.
Record for Q9CY34 not found.
Record for Q8VCZ6 not found.
Record for Q9D0F1 not found.
Record for Q80TG9 not found.
Record for Q8C7B8 not found.
Record for Q8R527 not found.
Record for Q99LL5 not found.
Record for Q60936 not found.
Record for Q3UUV5 not found.
Record for P63087 not found.
Record for P50592 not found.
Record for Q80Y84 not found.
Record for P62743 not found.
Record for Q61655 not found.
Record for Q9CQJ2 not found.
Record for Q99PM9 not found.
Record for Q80UG2 not found.
Record for Q8BZ03 not found.
Record for Q61627 not found.
Record for P08207 not found.
Record for Q9ERT9 not found.
Record for Q9D0K0 not found.
Record for Q9CQ20 not found.
Record for O09131 not found.
Record for Q8K285 not found.
Record for P62855 not found

Record for Q61033 not found.
Record for Q9EP97 not found.
Record for Q9CY21 not found.
Record for P45591 not found.
Record for P55200 not found.
Record for Q6PF93 not found.
Record for Q6P5F9 not found.
Record for P70697 not found.
Record for Q9DBK0 not found.
Record for P62515 not found.
Record for Q9CS72 not found.
Record for Q8BVG4 not found.
Record for P52793 not found.
Record for Q9CXU0 not found.
Record for P31996 not found.
Record for P28063 not found.
Record for Q60597 not found.
Record for Q3HVF0 not found.
Record for P14142 not found.
Record for Q9D6K9 not found.
Record for Q62048 not found.
Record for Q8BGJ9 not found.
Record for Q3UA06 not found.
Record for Q8BGD9 not found.
Record for Q99JH1 not found.
Record for Q6R891 not found.
Record for Q99MV2 not found.
Record for P04247 not found.
Record for P97329 not found.
Record for P41539 not found.
Record for Q64339 not found.
Record for Q6PHZ2 not found.
Record for Q8BND3 not found.
Record for Q8BX46 not found.
Record for Q6N

Record for O70302 not found.
Record for P10854 not found.
Record for A2AM29 not found.
Record for Q5DTY9 not found.
Record for Q8BR65 not found.
Record for Q6TYB5 not found.
Record for Q62159 not found.
Record for Q8CFT2 not found.
Record for Q99MR9 not found.
Record for Q01231 not found.
Record for O35235 not found.
Record for Q9D3D9 not found.
Record for P17156 not found.
Record for Q8BXQ2 not found.
Record for Q06180 not found.
Record for Q80TM6 not found.
Record for Q9Z0F3 not found.
Record for Q20271 not found.
Record for Q61096 not found.
Record for Q61824 not found.
Record for A9JP20 not found.
Record for P20490 not found.
Record for P09581 not found.
Record for P07309 not found.
Record for P15119 not found.
Record for Q80VL1 not found.
Record for P14100 not found.
Record for Q61423 not found.
Record for B1WC96 not found.
Record for Q9D9M2 not found.
Record for Q8BVZ5 not found.
Record for A2ARA8 not found.
Record for Q9CX00 not found.
Record for P15247 not found.
Record for O54

Record for O70165 not found.
Record for Q61263 not found.
Record for Q3TUD9 not found.
Record for A2F2E5 not found.
Record for Q9JMB7 not found.
Record for Q8CG76 not found.
Record for Q61699 not found.
Record for Q9EPC1 not found.
Record for Q99JP6 not found.
Record for Q9QYC3 not found.
Record for Q8R1S0 not found.
Record for Q9DBS8 not found.
Record for O49596 not found.
Record for Q9WV72 not found.
Record for Q9QXT8 not found.
Record for P34169 not found.
Record for P16872 not found.
Record for Q62401 not found.
Record for Q7TQH0 not found.
Record for Q9CZY2 not found.
Record for P56960 not found.
Record for O09061 not found.
Record for O70481 not found.
Record for Q6PAR5 not found.
Record for P30275 not found.
Record for Q9D8Y7 not found.
Record for Q9WTX2 not found.
Record for P48168 not found.
Record for Q9D8T4 not found.
Record for P26011 not found.
Record for Q9DBZ1 not found.
Record for P70120 not found.
Record for Q9CQH3 not found.
Record for Q9R000 not found.
Record for Q9D

Record for P10417 not found.
Record for P97343 not found.
Record for Q61456 not found.
Record for O70433 not found.
Record for Q03386 not found.
Record for P35761 not found.
Record for C3VPR6 not found.
Record for Q0VEJ0 not found.
Record for P07688 not found.
Record for Q9EPK5 not found.
Record for Q8CEE6 not found.
Record for P70121 not found.
Record for Q80ZK9 not found.
Record for Q9CQX2 not found.
Record for Q9DB07 not found.
Record for Q9ER47 not found.
Record for Q8VHR0 not found.
Record for Q6P5D8 not found.
Record for Q9WVS0 not found.
Record for P35821 not found.
Record for Q9Z1B5 not found.
Record for P26323 not found.
Record for O08810 not found.
Record for P46735 not found.
Record for Q9FHY3 not found.
Record for O54946 not found.
Record for P63212 not found.
Record for Q9CXV9 not found.
Record for P97386 not found.
Record for Q64385 not found.
Record for P81117 not found.
Record for P70696 not found.
Record for P26041 not found.
Record for Q8VHK5 not found.
Record for Q63

...

Record for Q9QYF1 not found.
Record for P58043 not found.
Record for P00514 not found.
Record for Q91V27 not found.
Record for Q8K199 not found.
Record for Q9Z1Y4 not found.
Record for P31725 not found.
Record for Q80U93 not found.
Record for Q9R0A0 not found.
Record for Q80XE1 not found.
Record for Q99J45 not found.
Record for Q3SYK4 not found.
Record for Q8C419 not found.
Record for Q9CQI3 not found.
Record for Q8R1F0 not found.
Record for P48759 not found.
Record for Q7TNP2 not found.
Record for Q3UZ39 not found.
Record for Q61205 not found.
Record for Q8K1I7 not found.
Record for Q61838 not found.
Record for Q5BL07 not found.
Record for Q8VIG1 not found.
Record for Q8R104 not found.
Record for O08784 not found.
Record for A2BH40 not found.
Record for Q9D1H9 not found.
Record for Q7TQJ8 not found.
Record for P47738 not found.
Record for Q9CQ33 not found.
Record for Q6NQH1 not found.
Record for O09053 not found.
Record for Q9WTR0 not found.
Record for Q2PZL6 not found.
Record for Q99

Record for Q9R1E6 not found.
Record for Q9CWS0 not found.
Record for P34960 not found.
Record for H2L0C1 not found.
Record for P82019 not found.
Record for Q61024 not found.
Record for Q9QXJ1 not found.
Record for P27577 not found.
Record for P63013 not found.
Record for Q9JK84 not found.
Record for O35239 not found.
Record for Q8BH97 not found.
Record for Q9Z2H7 not found.
Record for P62746 not found.
Record for P55066 not found.
Record for Q8BM55 not found.
Record for P20693 not found.
Record for Q3U0K8 not found.
Record for Q8K4K4 not found.
Record for Q921V5 not found.
Record for Q9EPX5 not found.
Record for Q8K3B1 not found.
Record for Q8QZX2 not found.
Record for Q8BL80 not found.
Record for Q8R1Q8 not found.
Record for Q9D6J6 not found.
Record for O70230 not found.
Record for Q5SXY1 not found.
Record for Q8BLG0 not found.
Record for Q925I1 not found.
Record for Q920G9 not found.
Record for Q99MB7 not found.
Record for Q8K4H1 not found.
Record for P59016 not found.
Record for Q07

Record for Q8BTS4 not found.
Record for P81067 not found.
Record for O88738 not found.
Record for Q8K0D0 not found.
Record for O89050 not found.
Record for Q7TQK9 not found.
Record for Q8BJH1 not found.
Record for P18265 not found.
Record for Q8CHT0 not found.
Record for Q9ES89 not found.
Record for Q9QXX0 not found.
Record for Q3UG98 not found.
Record for Q9EPN1 not found.
Record for Q8BK64 not found.
Record for P52843 not found.
Record for Q9VCQ8 not found.
Record for Q9R069 not found.
Record for Q3UW68 not found.
Record for Q61233 not found.
Record for O35130 not found.
Record for Q9DC28 not found.
Record for P03929 not found.
Record for Q80W85 not found.
Record for D4QAW2 not found.
Record for Q9D5T0 not found.
Record for P0A698 not found.
Record for Q6PNC0 not found.
Record for Q7TSI0 not found.
Record for Q60949 not found.
Record for Q2WF71 not found.
Record for P35700 not found.
Record for Q60805 not found.
Record for P97313 not found.
Record for Q9DCN1 not found.
Record for Q9R

Record for Q9CYZ8 not found.
Record for Q9WUU9 not found.
Record for Q64338 not found.
Record for Q7TNG5 not found.
Record for Q00977 not found.
Record for Q9WU19 not found.
Record for Q08851 not found.
Record for Q80Z19 not found.
Record for P43247 not found.
Record for Q99NB5 not found.
Record for Q64731 not found.
Record for Q7TQP4 not found.
Record for Q62086 not found.
Record for P45700 not found.
Record for P56392 not found.
Record for Q9WVS8 not found.
Record for P49194 not found.
Record for P55284 not found.
Record for Q9JHW2 not found.
Record for P67778 not found.
Record for P62492 not found.
Record for Q9QZB6 not found.
Record for Q99KK7 not found.
Record for Q99K43 not found.
Record for Q8BP86 not found.
Record for Q9CQC7 not found.
Record for Q9QXN5 not found.
Record for Q925F4 not found.
Record for Q96329 not found.
Record for Q9R224 not found.
Record for Q3TC72 not found.
Record for Q3E6Q1 not found.
Record for O92563 not found.
Record for O08811 not found.
Record for Q9J

Record for A2ABV5 not found.
Record for P51667 not found.
Record for Q80X44 not found.
Record for O35963 not found.
Record for Q80UK7 not found.
Record for Q8VI56 not found.
Record for P25119 not found.
Record for P28700 not found.
Record for P49764 not found.
Record for Q8R0I0 not found.
Record for P08228 not found.
Record for Q8CIP4 not found.
Record for Q61029 not found.
Record for P62774 not found.
Record for Q80U72 not found.
Record for Q9JKE2 not found.
Record for O70589 not found.
Record for Q9R0B6 not found.
Record for P40935 not found.
Record for Q04887 not found.
Record for Q8K1J5 not found.
Record for P02463 not found.


In [23]:
print("Mapping each network to the most recent uniprot accessions...")
kegg = map_network_accessions(
    interactions=kegg, accession_map=accession_mapping,
    drop_nan=True, allow_self_edges=True,
    allow_duplicates=False, min_counts=None, merge=False
)

In [24]:
hprd = map_network_accessions(
    interactions=hprd, accession_map=accession_mapping,
    drop_nan=True, allow_self_edges=True,
    allow_duplicates=False, min_counts=None, merge=False
)

In [25]:
pina2 = map_network_accessions(
    interactions=pina2, accession_map=accession_mapping,
    drop_nan=True, allow_self_edges=True,
    allow_duplicates=False, min_counts=None, merge=False
)

In [26]:
bioplex = map_network_accessions(
    interactions=bioplex, accession_map=accession_mapping,
    drop_nan=True, allow_self_edges=True,
    allow_duplicates=False, min_counts=None, merge=False
)

In [27]:
innate_c = map_network_accessions(
    interactions=innate_c, accession_map=accession_mapping,
    drop_nan=True, allow_self_edges=True,
    allow_duplicates=False, min_counts=None, merge=False
)

In [28]:
innate_i = map_network_accessions(
    interactions=innate_i, accession_map=accession_mapping,
    drop_nan=True, allow_self_edges=True,
    allow_duplicates=False, min_counts=None, merge=False
)
networks = [hprd, kegg, bioplex, pina2, innate_i, innate_c]

In [ ]:
print("Building features for each protein and PPI...")
ae = AnnotationExtractor(
    induce=True,
    selection=selection,
    n_jobs=3,
    verbose=True,
    cache=not update_features,
    backend='threading',
)

cond_1 = os.path.isfile(ppi_features_path)
cond_2 = os.path.isfile(accession_features_path)
if update_features or (not cond_1) or (not cond_2):
    tuple_gens = [zip(n.source, n.target) for n in networks]
    ppis = [(a, b) for tuples in tuple_gens for (a, b) in tuples]
    ae.fit(ppis)

    # Sanity check
    unique_ppis = set()
    unique_acc = set()
    for df in networks:
        ppis = list(zip(df.source, df.target))
        unique_ppis |= set(PPI(a, b) for (a, b) in ppis)
        unique_acc |= set(a for a, _ in ppis) | set(b for _, b in ppis)
    assert ae.accession_vocabulary.shape[0] == len(unique_acc)
    assert ae.ppi_vocabulary.shape[0] == len(unique_ppis)
    ae.cache()

Building features for each protein and PPI...
Acquiring data for accession 1/17927..
Acquiring data for accession 2/17927..
Acquiring data for accession 3/17927..
Acquiring data for accession 4/17927..
Acquiring data for accession 5/17927..
Acquiring data for accession 6/17927..
Acquiring data for accession 7/17927..
Acquiring data for accession 8/17927..
Acquiring data for accession 9/17927..
Acquiring data for accession 10/17927..
Acquiring data for accession 11/17927..
Acquiring data for accession 12/17927..
Acquiring data for accession 13/17927..
Acquiring data for accession 14/17927..
Acquiring data for accession 15/17927..
Acquiring data for accession 16/17927..
Acquiring data for accession 17/17927..
Acquiring data for accession 18/17927..
Acquiring data for accession 19/17927..
Acquiring data for accession 20/17927..
Acquiring data for accession 21/17927..
Acquiring data for accession 22/17927..
Acquiring data for accession 23/17927..
Acquiring data for accession 24/17927..
Acq

Acquiring data for accession 201/17927..
Acquiring data for accession 202/17927..
Acquiring data for accession 203/17927..
Acquiring data for accession 204/17927..
Acquiring data for accession 205/17927..
Acquiring data for accession 206/17927..
Acquiring data for accession 207/17927..
Acquiring data for accession 208/17927..
Acquiring data for accession 209/17927..
Acquiring data for accession 210/17927..
Acquiring data for accession 211/17927..
Acquiring data for accession 212/17927..
Acquiring data for accession 213/17927..
Acquiring data for accession 214/17927..
Acquiring data for accession 215/17927..
Acquiring data for accession 216/17927..
Acquiring data for accession 217/17927..
Acquiring data for accession 218/17927..
Acquiring data for accession 219/17927..
Acquiring data for accession 220/17927..
Acquiring data for accession 221/17927..
Acquiring data for accession 222/17927..
Acquiring data for accession 223/17927..
Acquiring data for accession 224/17927..
Acquiring data f

Acquiring data for accession 400/17927..
Acquiring data for accession 401/17927..
Acquiring data for accession 402/17927..
Acquiring data for accession 403/17927..
Acquiring data for accession 404/17927..
Acquiring data for accession 405/17927..
Acquiring data for accession 406/17927..
Acquiring data for accession 407/17927..
Acquiring data for accession 408/17927..
Acquiring data for accession 409/17927..
Acquiring data for accession 410/17927..
Acquiring data for accession 411/17927..
Acquiring data for accession 412/17927..
Acquiring data for accession 413/17927..
Acquiring data for accession 414/17927..
Acquiring data for accession 415/17927..
Acquiring data for accession 416/17927..
Acquiring data for accession 417/17927..
Acquiring data for accession 418/17927..
Acquiring data for accession 419/17927..
Acquiring data for accession 420/17927..
Acquiring data for accession 421/17927..
Acquiring data for accession 422/17927..
Acquiring data for accession 423/17927..
Acquiring data f

Acquiring data for accession 599/17927..
Acquiring data for accession 600/17927..
Acquiring data for accession 601/17927..
Acquiring data for accession 602/17927..
Acquiring data for accession 603/17927..
Acquiring data for accession 604/17927..
Acquiring data for accession 605/17927..
Acquiring data for accession 606/17927..
Acquiring data for accession 607/17927..
Acquiring data for accession 608/17927..
Acquiring data for accession 609/17927..
Acquiring data for accession 610/17927..
Acquiring data for accession 611/17927..
Acquiring data for accession 612/17927..
Acquiring data for accession 613/17927..
Acquiring data for accession 614/17927..
Acquiring data for accession 615/17927..
Acquiring data for accession 616/17927..
Acquiring data for accession 617/17927..
Acquiring data for accession 618/17927..
Acquiring data for accession 619/17927..
Acquiring data for accession 620/17927..
Acquiring data for accession 621/17927..
Acquiring data for accession 622/17927..
Acquiring data f

Acquiring data for accession 798/17927..
Acquiring data for accession 799/17927..
Acquiring data for accession 800/17927..
Acquiring data for accession 801/17927..
Acquiring data for accession 802/17927..
Acquiring data for accession 803/17927..
Acquiring data for accession 804/17927..
Acquiring data for accession 805/17927..
Acquiring data for accession 806/17927..
Acquiring data for accession 807/17927..
Acquiring data for accession 808/17927..
Acquiring data for accession 809/17927..
Acquiring data for accession 810/17927..
Acquiring data for accession 811/17927..
Acquiring data for accession 812/17927..
Acquiring data for accession 813/17927..
Acquiring data for accession 814/17927..
Acquiring data for accession 815/17927..
Acquiring data for accession 816/17927..
Acquiring data for accession 817/17927..
Acquiring data for accession 818/17927..
Acquiring data for accession 819/17927..
Acquiring data for accession 820/17927..
Acquiring data for accession 821/17927..
Acquiring data f

...

Acquiring data for accession 17417/17927..
Acquiring data for accession 17418/17927..
Acquiring data for accession 17419/17927..
Acquiring data for accession 17420/17927..
Acquiring data for accession 17421/17927..
Acquiring data for accession 17422/17927..
Acquiring data for accession 17423/17927..
Acquiring data for accession 17424/17927..
Acquiring data for accession 17425/17927..
Acquiring data for accession 17426/17927..
Acquiring data for accession 17427/17927..
Acquiring data for accession 17428/17927..
Acquiring data for accession 17429/17927..
Acquiring data for accession 17430/17927..
Acquiring data for accession 17431/17927..
Acquiring data for accession 17432/17927..
Acquiring data for accession 17433/17927..
Acquiring data for accession 17434/17927..
Acquiring data for accession 17435/17927..
Acquiring data for accession 17436/17927..
Acquiring data for accession 17437/17927..
Acquiring data for accession 17438/17927..
Acquiring data for accession 17439/17927..
Acquiring d

Acquiring data for accession 17607/17927..
Acquiring data for accession 17608/17927..
Acquiring data for accession 17609/17927..
Acquiring data for accession 17610/17927..
Acquiring data for accession 17611/17927..
Acquiring data for accession 17612/17927..
Acquiring data for accession 17613/17927..
Acquiring data for accession 17614/17927..
Acquiring data for accession 17615/17927..
Acquiring data for accession 17616/17927..
Acquiring data for accession 17617/17927..
Acquiring data for accession 17618/17927..
Acquiring data for accession 17619/17927..
Acquiring data for accession 17620/17927..
Acquiring data for accession 17621/17927..
Acquiring data for accession 17622/17927..
Acquiring data for accession 17623/17927..
Acquiring data for accession 17624/17927..
Acquiring data for accession 17625/17927..
Acquiring data for accession 17626/17927..
Acquiring data for accession 17627/17927..
Acquiring data for accession 17628/17927..
Acquiring data for accession 17629/17927..
Acquiring d

Acquiring data for accession 17797/17927..
Acquiring data for accession 17798/17927..
Acquiring data for accession 17799/17927..
Acquiring data for accession 17800/17927..
Acquiring data for accession 17801/17927..
Acquiring data for accession 17802/17927..
Acquiring data for accession 17803/17927..
Acquiring data for accession 17804/17927..
Acquiring data for accession 17805/17927..
Acquiring data for accession 17806/17927..
Acquiring data for accession 17807/17927..
Acquiring data for accession 17808/17927..
Acquiring data for accession 17809/17927..
Acquiring data for accession 17810/17927..
Acquiring data for accession 17811/17927..
Acquiring data for accession 17812/17927..
Acquiring data for accession 17813/17927..
Acquiring data for accession 17814/17927..
Acquiring data for accession 17815/17927..
Acquiring data for accession 17816/17927..
Acquiring data for accession 17817/17927..
Acquiring data for accession 17818/17927..
Acquiring data for accession 17819/17927..
Acquiring d

[Parallel(n_jobs=1)]: Done 17927 out of 17927 | elapsed:  1.1min finished


Computing selected features for each PPI...


In [ ]:
print("Saving raw networks...")
save_network_to_path(kegg, kegg_network_path)
save_network_to_path(hprd, hprd_network_path)
save_network_to_path(pina2, pina2_network_path)
save_network_to_path(bioplex, bioplex_network_path)
save_network_to_path(innate_i, innate_i_network_path)
save_network_to_path(innate_c, innate_c_network_path)

In [ ]:
print("Building and saving processed networks...")
hprd_test_labels = ['dephosphorylation', 'phosphorylation']
hprd_train_labels = set([l for l in hprd[LABEL] if l not in hprd_test_labels])
train_hprd = remove_labels(hprd, hprd_test_labels)
training = pd.concat([kegg, train_hprd], ignore_index=True)
testing = remove_intersection(remove_labels(hprd, hprd_train_labels), kegg)
full_training = pd.concat([training, testing], ignore_index=True)

testing = process_interactions(
    interactions=testing, drop_nan=True,
    allow_duplicates=False, allow_self_edges=True,
    exclude_labels=None, min_counts=5, merge=True
)
training = process_interactions(
    interactions=training,
    drop_nan=True, allow_duplicates=False, allow_self_edges=True,
    exclude_labels=None, min_counts=5, merge=True
)
full_training = process_interactions(
    interactions=full_training,
    drop_nan=True, allow_duplicates=False, allow_self_edges=True,
    exclude_labels=None, min_counts=5, merge=True
)

labels = list(training[LABEL]) + list(testing[LABEL])
ptm_labels = set(l for merged in labels for l in merged.split(','))
save_ptm_labels(ptm_labels)

interactome_networks = [bioplex, pina2, innate_i, innate_c]
interactome = pd.concat(interactome_networks, ignore_index=True)
interactome = process_interactions(
    interactions=interactome, drop_nan=True,
    allow_duplicates=False, allow_self_edges=True,
    exclude_labels=None, min_counts=None, merge=True
)
save_network_to_path(interactome, interactome_network_path)
save_network_to_path(training, training_network_path)
save_network_to_path(testing, testing_network_path)
save_network_to_path(full_training, full_training_network_path)